<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/missions_radio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CASDA, NRAO, ATNF (Radio-wave)**

![sciences](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_2000.png)

In [ ]:
!pip install astropy astroquery requests reproject -q
from astropy import units as u
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from astropy import coordinates
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
import numpy as np
import matplotlib.pyplot as plt
from astropy import coordinates as coords
from reproject import reproject_interp
import matplotlib.pyplot as plt

In [ ]:
# Some tasks like multi-wavelength astronomy require high memory to load data
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Runtime: {:.1f} gigabytes of available RAM (>20 GB ideal)\n'.format(ram_gb))

Runtime: 54.8 gigabytes of available RAM (>20 GB ideal)



#### **Define astronomical object and get skycoordinates**

In [ ]:
# Define object
object_name = 'M51'

*e.g. M31 (Andromeda galaxy), M104 (Sombrero galaxy), M51 (Whirlpool galaxy), M42 (Orion nebula), NGC 3372 (Carina nebula)*

In [ ]:
# Get and set skycoordinates of object
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord

skyobject = Simbad.query_object(object_name)
ra_hms = skyobject['RA'][0]
dec_dms = skyobject['DEC'][0]
coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
print(skyobject)
print("\nCoordinates:")
print("RA (h:m:s):", ra_hms)
print("DEC (d:m:s):", dec_dms)

MAIN_ID      RA          DEC      RA_PREC ... COO_WAVELENGTH     COO_BIBCODE     SCRIPT_NUMBER_ID
          "h:m:s"      "d:m:s"            ...                                                    
------- ------------ ------------ ------- ... -------------- ------------------- ----------------
  M  51 13 29 52.698 +47 11 42.93       7 ...              I 2006AJ....131.1163S                1

Coordinates:
RA (h:m:s): 13 29 52.698
DEC (d:m:s): +47 11 42.93


In [ ]:
# Convert from Right Ascension (RA) and Declination (DEC) to decimal degree
coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
ra_deg = coord.ra.deg # round(coord.ra.deg, 2)
dec_deg = coord.dec.deg # round(coord.dec.deg, 2)

print("RA in degrees:", ra_deg)
print("DEC in degrees:", dec_deg)

RA in degrees: 202.46957499999996
DEC in degrees: 47.19525833333333


In [ ]:
# Convert from decimal degree to Right Ascension (RA) and Declination (DEC)
coord = SkyCoord(ra=ra_deg, dec=dec_deg, unit='deg', frame='icrs')

# RA: hours, minutes, seconds. DEC: degrees, arcminutes, arcseconds
ra_hms = coord.ra.to_string(unit='hour', sep=' ', precision=3, pad=True)
dec_dms = coord.dec.to_string(unit='deg', sep=' ', precision=2, alwayssign=True, pad=True)

print("RA (h:m:s):", ra_hms)
print("DEC (d:m:s):", dec_dms)

RA (h:m:s): 13 29 52.698
DEC (d:m:s): +47 11 42.93


In [ ]:
# Reverse check to see to which object the coordinates belong
coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
result = Simbad.query_region(coord, radius='0d0m5s')  # Search within a small radius (5 arcseconds)
if result:
    print(result['MAIN_ID'][0])  # Show main name of the object
else:
    print("No object found in SIMBAD at these coordinates.")
object_name = 'MAIN_ID'

M  51


frame='icrs' refers to [International Celestial Ref. System](https://aa.usno.navy.mil/faq/ICRS_doc)
*  Origin is barycenter of Solar System. This means ICRS origin is not fixed to Earth but moves with Solar System's center of mass.
* Axes are defined by the positions of extragalactic sources, primarily quasars observed using Very Long Baseline Interferometry (VLBI). The primary plane is as close as possible to the mean equator at J2000.0 (January 1, 2000, 12:00 TT).  The origin of right ascension is close to the dynamical equinox of J2000.0.
* Accuracy: The ICRF3 defines the ICRS axes with an accuracy of about 20 microarcseconds.
* Realization: The International Celestial Reference Frame (ICRF) is a catalog of precise positions of extragalactic sources that provides a practical realization of the ICRS. The latest version, ICRF3, includes the positions of over 4000 extragalactic sources, mostly quasars. The ICRF is maintained by the International Earth Rotation and Reference Systems Service (IERS).
* Relationship to other systems: ICRS is closely aligned with the J2000 equatorial coordinate system, but it's more accurate and stable. Transformations exist to convert coordinates between ICRS and other celestial reference systems

In [ ]:
# Set coordinates manually and verify correct object
coord = SkyCoord('189.9976d -11.623054d', frame='icrs')
result = Simbad.query_region(coord, radius='0d0m5s')  # Search within a small radius (5 arcseconds)
if result:
    print(result['MAIN_ID'][0])  # Show main name of the object
else:
    print("No object found in SIMBAD at these coordinates.")
object_name = 'MAIN_ID'

M 104


#### **CASDA, NRAO, ATNF (Radio)**

CASDA Queries (astroquery.casda) - The CSIRO ASKAP Science Data Archive (CASDA) provides access to science-ready data products from observations at the [Australian Square Kilometre Array Pathfinder](https://casda.csiro.au/) (ASKAP) telescope. These data products include source catalogues, images, spectral line and polarisation cubes, spectra and visbilities. This package allows querying of the data products available in CASDA.

The Australian Square Kilometre Array Pathfinder (ASKAP) is a radio telescope array located at the Murchison Radio-astronomy Observatory in Western Australia. Here are its key features:

ASKAP consists of 36 identical antennas, each 12 meters in diameter, spread across a region 6 kilometers in diameter. What makes ASKAP particularly innovative is its use of phased array feed (PAF) technology - each antenna has a special receiver that can look at 36 different points on the sky simultaneously, dramatically increasing the telescope's survey speed.

The telescope operates in the frequency range of 700-1800 MHz and is designed to be a survey instrument, capable of mapping large areas of the sky quickly. It's particularly good at:
- Detecting and studying neutral hydrogen in galaxies
- Finding radio transients (brief radio bursts)
- Studying magnetic fields in space
- Discovering new galaxies

ASKAP is also significant because it's a precursor to the Square Kilometre Array (SKA), helping develop and test technologies that will be used in that larger project. It's already produced some important surveys like:
- The Rapid ASKAP Continuum Survey (RACS)
- The ASKAP Variables and Slow Transients (VAST) survey
- The Evolutionary Map of the Universe (EMU) survey

https://astroquery.readthedocs.io/en/latest/casda/casda.html

In [ ]:
from astroquery.casda import Casda

result_table = Casda.query_region(object_name, radius=30*u.arcmin) # SkyCoord.from_name('NGC 7232')
print(result_table['obs_publisher_did','s_ra', 's_dec', 'obs_release_date'][:5])

 obs_publisher_did         s_ra              s_dec            obs_release_date    
                           deg                deg                                 
------------------- ------------------ ------------------ ------------------------
 scan-391742-580173 10.684975329835696 41.486269663596545 2024-04-09T00:50:53.050Z
scan-557636-3428863 10.178423343160535  41.22225071160026                         
        cube-223628     10.42040693846     41.84135277084 2023-11-20T11:36:12.720Z
        cube-105254    10.420406938445    41.841352770837 2023-04-26T10:48:27.911Z
        cube-105255    10.420406938445    41.841352770837 2023-04-26T10:48:27.911Z


In [ ]:
from astropy import coordinates, units as u, wcs
from astroquery.casda import Casda
centre = coordinates.SkyCoord.from_name('2MASX J08161181-7039447')
casda = Casda()
casda.login(username='email@somewhere.edu.au')
result = Casda.query_region(centre, radius=30*u.arcmin)
public_data = Casda.filter_out_unreleased(result)
subset = public_data[((public_data['obs_collection'] == 'The Rapid ASKAP Continuum Survey') & #
                  (np.char.startswith(public_data['filename'], 'RACS-DR1_')) & #
                  (np.char.endswith(public_data['filename'], 'A.fits'))
                 )]
url_list = casda.cutout(subset[:1], coordinates=centre, radius=14*u.arcmin)
filelist = casda.download_files(url_list, savedir='/tmp')

NRAO (National Radio Astronomy Observatory) and ATNF (Australia Telescope National Facility)

In [ ]:
from astroquery.vo_conesearch import ConeSearch
from astropy.coordinates import SkyCoord
import astropy.units as u

# Query the NRAO VO service; catalog_db = 'https://archive.nrao.edu/vo/r2/NVSS'
result = ConeSearch.query_region(coord, radius=0.1*u.deg)
print(result)

    objID           gsc2ID      gsc1ID   hstID    ... multipleFlag compassGSC2id   Mag  
                                                  ...                              mag  
-------------- ---------------- ------ ---------- ... ------------ ------------- -------
23323175812944 00424433+4116085          2MJG1R2O ...            0 6453800072293   9.453
23323175812948 00424403+4116069          2MJG1R2S ...            0 6453800072297   9.321
23323175812933 00424455+4116103          2MJG1R2D ...            0 6453800072282  10.773
23323175812939 00424464+4116092          2MJG1R2J ...            0 6453800072288   9.299
23323175812930 00424403+4116108          2MJG1R2A ...            0 6453800072279  11.507
23323175812931 00424464+4116106          2MJG1R2B ...            0 6453800072280   9.399
23323175812973 00424446+4116016          2MJG1R3H ...            0 6453800072322   12.07
23323175812966 00424398+4116028          2MJG1R3A ...            0 6453800072315  12.136
23323175812979 004244

In [ ]:
print(result.info)

<Table length=4028>
     name      dtype    unit           description        
------------- ------- -------- ---------------------------
        objID   int64                          Database ID
       gsc2ID  object                              GSC2 ID
       gsc1ID  object                              GSC1 ID
        hstID  object                       GSC2 Public ID
           ra float64      deg      Right Ascension (ICRS)
          dec float64      deg          Declination (ICRS)
        epoch float32       yr         Epoch of coordinate
    raEpsilon float32   arcsec                    RA error
   decEpsilon float32   arcsec                   DEC error
         rapm float32 mas / yr            RA proper motion
        decpm float32 mas / yr           DEC proper motion
      rapmErr float32 mas / yr                 RA pm error
     decpmErr float32 mas / yr                DEC pm error
   deltaEpoch float32       yr   Time interval used for PM
       FpgMag float32      mag      

In [ ]:
from astroquery.atnf import Atnf

# Query the pulsar catalog
# You can specify parameters like period, distance, etc.
pulsar_table = Atnf.query_catalog(
    params=['JNAME', 'RAJ', 'DECJ', 'P0', 'DM'],
    condition='P0 < 0.1 && DM > 10'  # Example conditions
)

# Get info for a specific pulsar
pulsar = Atnf.get_pulsar('J0534+2200')  # Crab Pulsar